# Load the modules and continue from 7 steps later

In [121]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from pydap.client import open_url       # needed for OPeNDAP access
from pydap.cas.urs import setup_session # needed for Earthdata login

In [107]:
data = Dataset('spei01.nc')

#url = 'https://spei.csic.es/spei_database_2_7/nc/spei01.nc'
#dataset = open_url(url)

In [108]:
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Title: Global 1-month SPEI, z-values, 0.5 degree
    Version: 2.6
    Id: ./outputNcdf/spei01.nc
    Summary: Global dataset of the Standardized
	  Precipitation-Evapotranspiration Index (SPEI) at the 1-month time scale. Using CRU TS 4.05 precipitation and potential evapotranspiration data
    Keywords: drought, climatology, SPEI, Standardized
	  Precipitation-Evapotranspiration Index
    Institution: Consejo Superior de Investigaciones
	          Científicas, CSIC
    Url: http://sac.csic.es/spei
    Creators: Santiago Beguería <santiago.begueria@csic.es>
	          and Sergio Vicente-Serrano <svicen@ipe.csic.es>
    Software: Created in R using the SPEI package
	          (https://cran.r-project.org/web/packages/SPEI/;
	          https://github.com/sbegueria/SPEI)
    Call: spei.nc(sca=i, inPre=./inputData/cru_ts4.05.1901.2020.pre.dat.nc, outFile=paste)
    Date: Wed Apr 27 13:05:03 2022
    Ref

In [109]:
data.variables

{'lon': <class 'netCDF4._netCDF4.Variable'>
 float64 lon(lon)
     units: degrees_east
     long_name: longitude
     limits: -179.75, 179.75
     convention: Coordinates are referred to cell centres
 unlimited dimensions: 
 current shape = (720,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float64 lat(lat)
     units: degrees_north
     long_name: latitude
     limits: -89.75, 89.75
     convention: Coordinates are referred to cell centres
 unlimited dimensions: 
 current shape = (360,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'time': <class 'netCDF4._netCDF4.Variable'>
 float64 time(time)
     units: days since 1900-1-1
     long_name: time
     limits: 380, 11306
 unlimited dimensions: 
 current shape = (1440,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'spei': <class 'netCDF4._netCDF4.Variable'>
 float32 spei(time, lat, lon)
     units: z-values
     _FillValue: 1e+30
     lo

In [110]:
# lat = data.variables['lat'][:].data
# lon = data.variables['lon'][:].data
# time = data.variables['time'][:].data
# spei = data.variables['spei'][:].data
# spei_mask = data.variables['spei'][:].mask

lat1 = data.variables['lat'][180:300].data #getting a smaller region
lon1 = data.variables['lon'][480:600].data #getting a smaller region
time1 = data.variables['time'][1400:].data #filtering data for 2018 to 2020
spei1 = data.variables['spei'][1400:,180:300,480:600].data
fillvalue1 = data.variables['spei']._FillValue

In [111]:
spei1.shape, lat1.shape, lon1.shape

((40, 120, 120), (120,), (120,))

In [112]:
#storing the data so it works without spei01.nc file

np.savez("spei_analysis.npz", spei1, lat1, lon1, time1, fillvalue1)


## Start from here, if SPEI file is not in local.


In [120]:
# loading the data from disk

data_coo = np.load("spei_analysis.npz") 

In [113]:
#reloading the data

spei = data_coo['arr_0']
lat = data_coo['arr_1']
lon = data_coo['arr_2']
time = data_coo['arr_3']
fillvalue = data_coo['arr_4']

In [114]:
spei[spei==fillvalue] = np.nan   # set missing values or land areas to nan
data.close()
lenTime = len(spei[:,0,0])
lenTime = len(time)
lenLat = len(lat)
lenLon = len(lon)

In [115]:
# reshapes 3D data array to 2D big data matrix X:
X = spei.reshape((lenTime,lenLon*lenLat))
X = np.transpose(X) # transpose so that spatial components on rows and temporal measurement on columns of X
print('Size of X: ',X.shape)

Size of X:  (14400, 40)


In [116]:
# Find and save index location of nan/notnan values in X (land areas) in a 1D array
#    Note: will do this only for the first column/time step, 
#    because mask is the same for every time step
Xnan = np.where(np.isnan(X[:,0]))[0]    # index location of nan values
Xnotnan= np.where(~np.isnan(X[:,0]))[0] # index location of non-nan values 

print('Number of nan/land values in one column of X: ',len(Xnan))
print('Number of not-nan/ocean values in one column of X: ',len(Xnotnan))
print('\nIndex location of nan values:')
Xnan

Number of nan/land values in one column of X:  3899
Number of not-nan/ocean values in one column of X:  10501

Index location of nan values:


array([   0,    1,    2, ..., 9358, 9359, 9479], dtype=int64)

In [117]:
# Delete nan/land values from X in all columns, store in X1
X1 = X[Xnotnan] # filter out land values, via fancy indexing !
# note: Xnotnan indicees are broadcasted along time axis
print('Size of X1: ', X1.shape) 

Size of X1:  (10501, 40)


In [118]:
#Perform SVD with nan-filtered big matrix X1
u1,s,vT = np.linalg.svd(X1,full_matrices=0) # performs the singular value decomposition

LinAlgError: SVD did not converge

In [119]:
X1

array([[ 1.8981215 ,  0.19408588,  2.0734596 , ..., -0.6463183 ,
         0.25951445, -1.2417376 ],
       [ 1.8429812 ,  0.24117276,  2.1603158 , ..., -0.5126354 ,
         0.53261286, -1.1727356 ],
       [ 1.7374835 ,  0.21153432,  2.2121103 , ..., -0.46326703,
         0.79625475, -1.1162246 ],
       ...,
       [ 1.3805453 , -0.22022094,  2.1484227 , ..., -0.14402246,
         0.7718914 ,  0.8545277 ],
       [ 1.406701  ,  0.12592734,  2.23665   , ..., -0.25083083,
         0.8775608 ,  0.88728213],
       [ 1.4412843 , -0.18764895,  2.3420951 , ..., -0.12677708,
         0.99900806,  0.9777134 ]], dtype=float32)